<a href="https://colab.research.google.com/github/JPA-BERT/jpa-bert.github.io/blob/master/notebooks/2020_0723pytorch_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PyTorch tutorial の内容について

[https://github.com/pytorch/tutorials/tree/master/beginner_source/nlp](https://github.com/pytorch/tutorials/tree/master/beginner_source/nlp) を見ると
PyTorch で 自然言語処理を行う場合のチュートリアルは以下とおりである

# Deep Learning for NLP with Pytorch

1. [pytorch_tutorial.py](https://github.com/pytorch/tutorials/blob/master/beginner_source/nlp/pytorch_tutorial.py): 
	[PyTorch 入門 Introduction to PyTorch](https://pytorch.org/tutorials/beginner/nlp/pytorch_tutorial.html)

2. [deep_learning_tutorial.py](https://github.com/pytorch/tutorials/blob/master/beginner_source/nlp/deep_learning_tutorial.py): 
	[PyTorch による深層学習 Deep Learning with PyTorch](https://pytorch.org/tutorials/beginner/nlp/deep_learning_tutorial.html)

3. [word_embeddings_tutorial.py](https://github.com/pytorch/tutorials/blob/master/beginner_source/nlp/word_embeddings_tutorial.py): 
	[単語埋め込み:語彙的意味の符号化 Word Embeddings: Encoding Lexical Semantics](https://pytorch.org/tutorials/beginner/nlp/word_embeddings_tutorial.html)

4. [sequence_models_tutorial.py]((https://github.com/pytorch/tutorials/blob/master/beginner_source/nlp/sequence_models_tutorial.py): 
	[系列モデルと LSTM Sequence Models and Long-Short Term Memory Networks](https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html)

5. [advanced_tutorial.py]((https://github.com/pytorch/tutorials/blob/master/beginner_source/nlp/advanced_tutorial.py): 
	[動的意思決定と双方向 LSTM 条件付き確率場 Advanced: Making Dynamic Decisions and the Bi-LSTM CRF](https://pytorch.org/tutorials/beginner/nlp/advanced_tutorial.html)


以下では，このうちの 1 について解説している。


In [2]:
%matplotlib inline

---
- original: https://colab.research.google.com/github/pytorch/tutorials/blob/gh-pages/_downloads/7e00f82429a7b691e42bb5828dbb036e/pytorch_tutorial.ipynb
- 注: この colab ファイルは PyTorch の公式チュートリアルからの翻訳に手を加えたものです。
- date: 2020-0723
---

# Torch のテンソルライブラリ紹介 <!---Introduction to Torch's tensor library--->

テンソルとは 2 次以上の次元を持つ行列の一般化であり，ディープラーニングモデルはすべてテンソルに関する計算です。
これが何を意味するのかについては，後ほど詳しく説明します．まず，テンソルで何ができるかを見てみましょう．
<!---
All of deep learning is computations on tensors, which are generalizations of a matrix that can be indexed in more than 2 dimensions. 
We will see exactly what this means in-depth later. First, lets look what we can do with tensors.
-->


In [3]:
# Author: Robert Guthrie

import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

### テンソルの作成 

テンソルは `torch.tesor()` 関数を使って作成される Python のリストです。
<!--Tensors can be created from Python lists with the torch.tensor() function.-->




In [4]:
# torch.tensor(data) により任意のデータを持つ  torch.Tensor オブジェクトが生成されます。 
V_data = [1., 2., 3.]
V = torch.tensor(V_data)
print(V)

# 行列を生成します 
M_data = [[1., 2., 3.], [4., 5., 6]]
M = torch.tensor(M_data)
print(M)

# 2ｘ2ｘ2 の 3階テンソルを作成します
T_data = [[[1., 2.], [3., 4.]],
          [[5., 6.], [7., 8.]]]
T = torch.tensor(T_data)
print(T)

tensor([1., 2., 3.])
tensor([[1., 2., 3.],
        [4., 5., 6.]])
tensor([[[1., 2.],
         [3., 4.]],

        [[5., 6.],
         [7., 8.]]])


ここで  3 階テンソルとは何かについて考えてみます。
ベクトルであれば，ベクトルについた添字 (たとえば $\mathbf{x}_i$ における $i$) は，個々の要素すなわちスカラとなります。
行列であれば，行列につけられた添字 ($\mathbf{X}_i$) はベクトルとなります。（訳注，2 つの添字 $\mathbf{X}_{ij}$ でスカラとなる）
3 次元テンソル (3 階テンソル) であれば、テンソルにつけられた添字は行列になります。

用語の注意：
このチュートリアルで「テンソル」と言うときは、任意の `torch.Tensor` オブジェクトを指します。
行列とベクトルは `torch.Tensors` の特殊なケースであり，次元はそれぞれ 1 と 2 です。
3 次元テンソルについて話すときは、明示的に "3次元テンソル "という用語を使います。



<!--What is a 3D tensor anyway? Think about it like this. 
If you have a vector, indexing into the vector gives you a scalar. 
If you have a matrix, indexing into the matrix gives you a vector. 
If you have a 3D  tensor, then indexing into the tensor gives you a matrix!

A note on terminology:
when I say "tensor" in this tutorial, it refers to any `torch.Tensor` object. 
Matrices and vectors are special cases of `torch.Tensors`, where their dimension is 1 and 2 respectively. 
When I am talking about 3D tensors, I will explicitly use the term "3D tensor".
-->



In [5]:
# Index into V and get a scalar (0 dimensional tensor)
print(V[0])
# Get a Python number from it
print(V[0].item())

# Index into M and get a vector
print(M[0])

# Index into T and get a matrix
print(T[0])

tensor(1.)
1.0
tensor([1., 2., 3.])
tensor([[1., 2.],
        [3., 4.]])


他のデータ型のテンソルを作成することもできます。
整数型のテンソルを作るには、``torch.tensor([[1, 2], [3, 4]])`` を試してみてください。
また、``dtype=torch.data_type`` を渡すことでデータ型を指定することもできます。
その他のデータ型についてはドキュメントをチェックしてください。


<!--You can also create tensors of other data types. 
To create a tensor of integer types, try torch.tensor([[1, 2], [3, 4]]) (where all elements in the list are integers).
You can also specify a data type by passing in ``dtype=torch.data_type``.
Check the documentation for more data types, but Float and Long will be the most common.
-->



乱数でできたテンソルを作成するには `torch.randn()` を使ってください。
(訳注: Numpy で正規乱数を発生させるには `np.random.randn()` である。この場合，与える引数は 1 つであり，生成する乱数の数である。
テンソルに変換する際には `.reshape()` を用いる。`np.random.randn(3 * 4 * 5).reshape(3,4,5)` )
<!--
You can create a tensor with random data and the supplied dimensionality with `torch.randn()`
-->




In [13]:
x = torch.randn((3, 4, 5))
print(x)

tensor([[[ 0.4100,  0.4085,  0.2579,  1.0950, -0.5065],
         [ 0.0998, -0.6540,  0.7317, -1.4567,  1.6089],
         [ 0.0938, -1.2597,  0.2546, -0.5020, -1.0412],
         [ 0.7323,  1.3075, -1.1628,  0.1196, -0.1631]],

        [[ 0.6614,  1.1899,  0.8165, -0.9135, -0.3538],
         [ 0.7639, -0.5890, -0.7636,  1.3352,  0.6043],
         [-0.1034, -0.1512,  1.2466,  0.5057,  0.9505],
         [ 1.2966,  0.8738, -0.5603,  1.2858,  0.8168]],

        [[-1.4648, -1.2629,  1.1220,  1.5663, -1.0371],
         [-1.0669, -0.2085, -0.2155,  0.2705,  0.5597],
         [-0.3184,  1.5117, -1.5208,  0.9196, -0.5484],
         [-0.3472, -0.7474, -0.9234,  0.5734, -0.1093]]])


### テンソルの演算
<!--Operations with Tensors-->

<!--~~~~~~~~~~~~~~~~~~~~~~~

You can operate on tensors in the ways you would expect.
-->


In [14]:
x = torch.tensor([1., 2., 3.])
y = torch.tensor([4., 5., 6.])
z = x + y
print(z)

tensor([5., 7., 9.])


利用可能な膨大な数の操作の完全なリストは [`ドキュメント`](https://pytorch.org/docs/torch.html) を参照してください。
これらの操作は数学的な操作だけではありません。

後で使うことになる便利な操作の一つが 連結 `concatenate()` です。

<!--
See [`the documentation`](https://pytorch.org/docs/torch.html) for a complete list of the massive number of operations available to you. 
They expand beyond just mathematical operations.

One helpful operation that we will make use of later is concatenation.
-->



In [18]:
# デフォルトでは，最初の次元にそって連結されます
x_1 = torch.randn(2, 5)  # 2 行 5 列の行列 x_1 
y_1 = torch.randn(3, 5)  # 3 行 5 列の行列 y_1
z_1 = torch.cat([x_1, y_1])  # 連結すると 5 行 5 列の行列を得る
print(z_1)

# 列に沿った連結 Concatenate columns:
x_2 = torch.randn(2, 3)  # 2 行 3 列の行列 x_2
y_2 = torch.randn(2, 5)  # 2 行 5 列の行列 y_2
# 第 2 引数で，連結する次元を指定する 0 は 行，1 なら 列
z_2 = torch.cat([x_2, y_2], 1)
print(z_2)

# 2 つの行列が互換しない場合には，torch はエラーを吐く。下行をアンコメントしてエラーを確認してください
# torch.cat([x_1, x_2])

tensor([[ 1.1296,  0.2214, -0.0558,  1.2057,  1.9486],
        [-0.0766, -0.8562, -0.7870, -0.8161,  0.5470],
        [-1.1707, -0.4699, -1.6271, -0.1127,  1.5980],
        [-0.8445, -1.0489,  0.9387,  0.5378,  1.5372],
        [-0.6943,  0.2174, -0.2995, -0.3749,  1.8673]])
tensor([[ 0.9042,  0.1181,  1.8941,  1.1366, -1.9280, -0.5565, -0.1434,  2.6780],
        [-0.4229,  0.7431,  0.0756,  0.6735,  0.4161, -1.0110,  0.5515,  0.9910]])


## テンソルの形状変更 <!--Reshaping Tensors-->

テンソルの形状を変更するには `.view()` メソッドを使用します。
多くのニューラルネットワークコンポーネントは入力が特定の形状を持つことが仮定されています。
このため `.vew()` メソッドは頻繁に使用されます。
多くの場合 データをコンポーネントに渡す前に `.view()` で形状変更する必要があります。

<!--
Use the `.view()` method to reshape a tensor. 
This method receives heavy use, because many neural network components expect their inputs to have a certain shape. Often you will need to reshape before passing your data to the component.
-->



In [19]:
x = torch.randn(2, 3, 4)
print(x)
print(x.view(2, 12))  # 2 行 12 列の行列に形状変更 
# 上と等価な操作が次行。ある次元が -1 であれば，形状を適宜類推してくれる
print(x.view(2, -1))

tensor([[[-0.3884, -0.0316, -0.5606, -0.6555],
         [ 0.7262,  0.6789, -0.4302, -0.3849],
         [ 0.7148,  0.3969,  0.1927,  0.3429]],

        [[ 1.4161,  0.4738,  0.0827, -1.3034],
         [ 0.7252, -0.5464, -0.8024,  0.0186],
         [ 1.1349,  0.8658,  0.6334, -0.5392]]])
tensor([[-0.3884, -0.0316, -0.5606, -0.6555,  0.7262,  0.6789, -0.4302, -0.3849,
          0.7148,  0.3969,  0.1927,  0.3429],
        [ 1.4161,  0.4738,  0.0827, -1.3034,  0.7252, -0.5464, -0.8024,  0.0186,
          1.1349,  0.8658,  0.6334, -0.5392]])
tensor([[-0.3884, -0.0316, -0.5606, -0.6555,  0.7262,  0.6789, -0.4302, -0.3849,
          0.7148,  0.3969,  0.1927,  0.3429],
        [ 1.4161,  0.4738,  0.0827, -1.3034,  0.7252, -0.5464, -0.8024,  0.0186,
          1.1349,  0.8658,  0.6334, -0.5392]])


# 計算グラフ と自動微分 

計算グラフの概念は、効率的な深層学習プログラミングに不可欠です。
計算グラフは，データがどのように結合されて出力されるかを簡単に記述したものです．
グラフはどのパラメータがどの操作に関与したかを完全に特定しているので、導関数を計算するのに十分な情報を含んでいます。
曖昧に聞こえるかもしれませんが、基本フラグ ``requires_grad`` を使って何が起こっているのか見てみましょう。

まず、プログラマの立場から考えてみましょう。
先ほど作成した `torch.Tensor` オブジェクトには何が格納されているでしょうか？
明らかにデータと形状，そして他にもいくつかあるかもしれません。
しかし、2 つのテンソルを足し合わせると 出力テンソルが得られます。
この出力テンソルが知っているのはデータと形状だけです。
それが他の2つのテンソルの和であることは知りません
（ファイルから読み込まれたのかもしれませんし、他の操作の結果かもしれません）。

``requires_grad=True`` の場合，テンソルオブジェクトはどのようにして作られたかを追跡します。
実際に見てみましょう。

<!--
The concept of a computation graph is essential to efficient deep learning programming, because it allows you to not have to write the back propagation gradients yourself. 
A computation graph is simply a specification of how your data is combined to give you the output. 
Since the graph totally specifies what parameters were involved with which operations, it contains enough information to compute derivatives. 
This probably sounds vague, so let's see what is going on using the fundamental flag ``requires_grad``.

First, think from a programmers perspective. 
What is stored in the torch.Tensor objects we were creating above? 
Obviously the data and the shape, and maybe a few other things. But when we added two tensors together, we got an output tensor. 
All this output tensor knows is its data and shape. It has no idea that it was the sum of two other tensors (it could have been read in from a file, it could be the result of some other operation, etc.)

If ``requires_grad=True``, the Tensor object keeps track of how it was created. Lets see it in action.
-->



In [21]:
# PyTorch のテンソル操作には ``requries_grad`` フラグがあります
x = torch.tensor([1., 2., 3], requires_grad=True)

# requires_grad=True であれば，それ以前に行った演算操作を逆行できます 
y = torch.tensor([4., 5., 6], requires_grad=True)
z = x + y
print(z)

# ですが z には追加情報必要 BUT z knows something extra.
print(z.grad_fn)

tensor([5., 7., 9.], grad_fn=<AddBackward0>)


``z`` はそれがファイルから読み込まれたものではなく，乗算や指数などの結果でもないことを知っています。
そして ``z.grad_fn`` を辿れば ``x`` と ``y`` にたどり着きます。

では，どうやって勾配を計算するのに役立つのでしょうか？

<!--
So Tensors know what created them. z knows that it wasn't read in from a file, it wasn't the result of a multiplication or exponential or whatever. 
And if you keep following z.grad_fn, you will find yourself at x and y.

But how does that help us compute a gradient?
-->



In [22]:
# Lets sum up all the entries in z
s = z.sum()
print(s)
print(s.grad_fn)

tensor(21., grad_fn=<SumBackward0>)


So now, what is the derivative of this sum with respect to the first component of x? In math, we want
では，上記の和の微分で ``x`` の第一成分に対する微分は何でしょうか？ 数学的には:

\begin{align}
\frac{\partial s}{\partial x_0}
\end{align}

``s`` はテンソル ``z`` の要素の和から作られたことを知っています。
さらに ``z`` は，各要素が ``x + y`` から作られたことを知っています。従って:
<!--Well, s knows that it was created as a sum of the tensor z. z knows that it was the sum x + y. So-->

\begin{align}
s = \overbrace{x_0 + y_0}^\text{$z_0$} + \overbrace{x_1 + y_1}^\text{$z_1$} + \overbrace{x_2 + y_2}^\text{$z_2$}
\end{align}

s は，微分を定めるための十分な情報を保持していおり，答え 1 を得ることができます。
<!--And so s contains enough information to determine that the derivative we want is 1!-->

自動微分により，微分を実際に計算する方法の問題が隠蔽されます。
ここで重要ななことは ``s`` が微分を計算可能にする情報を保持しているということです。
実際には Pytorch での開発では sum() と + 演算をプログラムして それらの勾配を計算する方法を知り、
逆伝播アルゴリズムを実行します。
このアルゴリズムの詳細な議論は このチュートリアルの範囲を超えています。

<!--Of course this glosses over the challenge of how to actually compute that derivative. The point here is that s is carrying along enough information that it is possible to compute it. 
In reality, the developers of Pytorch program the sum() and + operations to know how to compute their gradients, and run the back propagation algorithm. 
An in-depth discussion of that algorithm is beyond the scope of this tutorial.
-->



Pytorch に勾配を計算させて，正しく動作することを確認してみましょう。
(このブロックを何度も実行すると 勾配が増加します。
これは Pytorch が勾配を  *累積* し `.grad` プロパティに格納しているためです。
多くのモデルではこの累積が非常に便利だからです)。


<!--Lets have Pytorch compute the gradient, and see that we were right:
(note if you run this block multiple times, the gradient will increment.
That is because Pytorch *accumulates* the gradient into the .grad property, since for many models this is very convenient.)-->




In [23]:
# calling .backward() on any variable will run backprop, starting from it.
s.backward()
print(x.grad)

tensor([1., 1., 1.])


下のセルで起こっていることを理解することがディープラーニングのプログラミングには非常に重要です。

<!--Understanding what is going on in the block below is crucial for being a successful programmer in deep learning.-->





In [26]:
x = torch.randn(2, 2)
y = torch.randn(2, 2)
# デフォルトでは，ユーザが作成したテンソル逆向き勾配計算はしない設定 ``requrires_grad=False`` です
print(x.requires_grad, y.requires_grad)
z = x + y
# z  を逆伝播させることができません。
print(z.grad_fn)

# ``.requires_grad_( ... )`` はテンソルが持つ ``requires_grad`` の値を変化させます。存在するchanges an existing Tensor's ``requires_grad``
# インプレース，すなわち変数の値を直接書き換えます。(訳注:多く PyTorch メソッドではアンダースコアがあるとインプレースでの変換になる)
# デフォルトでは ``Ture`` が設定される
x = x.requires_grad_()
y = y.requires_grad_()
# 上で見たように z は勾配計算のための情報を保持している
z = x + y
print(z.grad_fn)
# もしある操作に対する任意の入力で ``requrires_grad=True`` ではれば出力は以下のように成る
print(z.requires_grad)

# z は計算履歴をもち，x と y と関係づけられている。
# この値を取り出すには **detatch** して履歴から切り離す
new_z = z.detach()

# 上の ``new_z`` は逆向計算のための情報を保持敷いていのだろうか？
# 否! 
print(new_z.grad_fn)
# ``z.detach()`` は記憶装置に保持されているテンソル z と同じ内容を返す。
# だが，計算履歴は忘却する。どのようにして計算されたのかという情報を保持していない
# すなわち，`.detach()` メソッドはそのテンソルの過去の履歴を破壊する

False False
None
True
None


``with torch.no_grad():`` ブロックで包むことで，`requrires_grad=True` としてあったテンソルの履歴追跡を止めることができる。

<!--You can also stop autograd from tracking history on Tensors with ``.requires_grad``=True by wrapping the code block in ``with torch.no_grad():``
-->


In [27]:
print(x.requires_grad)
print((x ** 2).requires_grad)

with torch.no_grad():
	print((x ** 2).requires_grad)

True
True
False
